In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

- 매번 데이터를 불러오는데 데이터 분석을 하는데 보다 시간을 오래 쏟다 보니 그냥 google colab를 이용하는 김에 google drive에서 내 데이터를 불러오는 방법이 없을까 찾아보았다.
- ```from google.colab import drive
     drive.mount('/content/mount')```
  라고 입력을 하게 되면 드라이브의 문서와 연동이 되어서 지정한 directory에 모든 문서가 저장이 된다.

In [2]:
import os
from google.colab import drive
drive.mount('/content/mount')

Drive already mounted at /content/mount; to attempt to forcibly remount, call drive.mount("/content/mount", force_remount=True).


1. 데이터 불러오기

In [3]:
data = pd.read_csv('/content/mount/My Drive/twcs.csv')

In [4]:
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [5]:
df = data[['text']]

In [6]:
df['text'] = df['text'].astype(str)
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


#### 2. Lower Casing(대문자 <-> 소문자)

- 이 방법은 주로 어떤 데이터나 문서 속에서 단어의 반복 횟수 등을 고려해 주어야 할 때 많이 사용한다.
  - 단어의 반복/사용 횟수는 나중에 embedding할 때에 단어 벡터를 만드는데 필요하다.
- 그러나 고유 명사와 일반 명사를 다룰 때에 조심할 필요가 있다.
- vectorizer이나 tokenizer을 위해서 ```sklearn TfidVectorizer```이나 ```Keras Tokenizer```을 많이 이용한다.

In [7]:
df['lower_text'] = df['text'].str.lower()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
df.head()

,text,lower_text
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.


- 위에서 실행한 결과를 보면 알겠지만 'lower_text'라고 index가 붙은 경우의 내용은 모든 대문자가 소문자로 바뀌어있다.
#### 이제 시험 삼아서 나중에 데이터 전처리를 할 때에 자주 사용하게 될 단어의 벡터화를 저장된 모듈을 이용해서 연습하자.
  - sklearn의 tokenizer을 사용해 보자.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
c_vectorizer = CountVectorizer()
t_vectorizer = TfidfVectorizer()
temp = df['text'].values

In [10]:
temp[0]

'@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.'

- 중요한 것은 여기서 fit_transform()에 넣을 수 있는 데이터는 리스크에 들어있는 문자열의 연속이다.

In [11]:
print(c_vectorizer.fit_transform([temp[0],temp[1],temp[2],temp[3]]).toarray(), t_vectorizer.fit_transform([temp[0], temp[1], temp[2], temp[3]]).toarray())

[[1 0 0 2 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
  3 0 1 0 0 1 2 2 0]
 [0 1 0 0 0 0 0 2 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0
  0 0 0 0 0 1 0 1 0]
 [0 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 1 1 0 1 0 0
  0 0 0 0 1 0 0 0 0]
 [1 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 2 0 0 0 1 0 1 1 1 0 0 0 1 0 0 1 0 1 1
  0 1 0 1 0 1 0 1 1]] [[0.15410536 0.         0.         0.30821072 0.         0.
  0.         0.         0.15410536 0.19546311 0.         0.
  0.19546311 0.         0.         0.19546311 0.19546311 0.
  0.         0.19546311 0.         0.         0.         0.
  0.12476157 0.         0.         0.         0.19546311 0.
  0.         0.         0.         0.         0.         0.
  0.58638932 0.         0.19546311 0.         0.         0.12476157
  0.39092622 0.24952314 0.        ]
 [0.         0.26761048 0.         0.         0.         0.
  0.         0.67885993 0.         0.         0.         0.33942996
  0.         0.         0. 

- 두 vectorizer의 fit_trandform한 결괏값이 다름을 확인 할 수 있다. 
  1. CountVectorizer의 경우에는 각 문자열 별로 개수에 따라서 벡터를 구성하였다면
  2. TfidVectorizer의 경우에는 각 문자열 별로 개수는 세지만 확률값, 즉 0과 1 사이의 값으로 벡터를 구성하였다.

In [12]:
print(c_vectorizer.get_feature_names())
print(t_vectorizer.get_feature_names())

['115712', 'and', 'as', 'assist', 'at', 'can', 'click', 'do', 'further', 'get', 'have', 'how', 'into', 'is', 'just', 'like', 'link', 'message', 'messages', 'need', 'no', 'of', 'one', 'please', 'private', 'profile', 'propose', 'responding', 'secured', 'send', 'sent', 'several', 'so', 'sprintcare', 'that', 'the', 'to', 'top', 'understand', 'us', 'usual', 'we', 'would', 'you', 'your']
['115712', 'and', 'as', 'assist', 'at', 'can', 'click', 'do', 'further', 'get', 'have', 'how', 'into', 'is', 'just', 'like', 'link', 'message', 'messages', 'need', 'no', 'of', 'one', 'please', 'private', 'profile', 'propose', 'responding', 'secured', 'send', 'sent', 'several', 'so', 'sprintcare', 'that', 'the', 'to', 'top', 'understand', 'us', 'usual', 'we', 'would', 'you', 'your']


#### 3. Removal of Punctuations
- 예를 들면 우리는 'beautiful'과 'beautiful!'을 같은 단어로 취급하고 싶다.
- 만약에 python에 있는 ```string.punctuation```을 이용하게 되면 

!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`

이 값들을 punctiation으로서 인식하고 제거가 가능하다.
  - 필요에 따라서 우리가 punctuation으로 인식할 값들을 추가하거나 제거해 주어야 한다.

In [13]:
import string
punct_to_remove = string.punctuation
def remove_punctuation(text):
  return text.translate(str.maketrans('','',punct_to_remove))
df['text_to_punct'] = df['text'].apply(lambda text:remove_punctuation(text))
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,text,lower_text,text_to_punct
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 I understand I would like to assist you...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare I have sent several private message...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 Please send us a Private Message so tha...
4,@sprintcare I did.,@sprintcare i did.,sprintcare I did


- 모든 구두점등이 사라져있음을 알 수 있다.

In [14]:
punct_to_remove

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

#### 4. Removal of Stopwords
- 한글에서의 일부 조사처럼 우리가 사용하는 말에서 의미에 별로 영향을 주지 않는 단어들이 존재하는데, 이를 stopwords라고 부른다.
- 우리는 이값들을 제거해보고자 하는데, 이 단어들의 모음 또한 ```nltk.corpus```에서 import 할 수 있다.

In [15]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [20]:
file = set(stopwords.words('english'))
def remove_stopwords(text):
  return " ".join([word for word in str(text).split() if word not in file])

In [21]:
df['remove_stopword'] = df['text_to_punct'].apply(lambda text: remove_stopwords(text))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
df.head()

,text,lower_text,text_to_punct,remove_stopword
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...
4,@sprintcare I did.,@sprintcare i did.,sprintcare I did,sprintcare I


- 불필요한 조사및 어구들이 제거되었음을 알 수 있다.
- 이렇게 제거가 되어도 문장의 의미 자체에는 큰 차이가 없기 때문에 일반적으로 이렇게 제거를 해주는 것이 텍스트 전처리에서 중요하다.
  - 우리가 HTML형식등 다양한 현태의 문서를 text로 변경해야 하는 이유는 일반적으로 문장/문자열 전처리가 텍스트 형식일 때 쉽기 때문이다.

#### 5. Removal of Frequent Words
- 이 과정은 이전에 이메일 텍스트 전처리 할 때에도 중요했던 부분이다.
- 지금까지 진행한 전처리 과정은 미리 정해진 제거 후보의 단어 리스트를 이용했었다.
- 그러나 우리는 데이터 상황에 따라서 다른 선택을 해야 할 것인데, 이는 그중 한가지 방법이다.
  - 이번에는 제일 자주 등장하는 단어를 제거해 본다.
  - tfidf를 이용하면 쉽게 제거가 가능하다.

In [30]:
from collections import Counter
cnt = Counter()
for text in df['remove_stopword'].values:
  for word in text.split():
    cnt[word] += 1

In [25]:
cnt.most_common(10)

[('I', 833982),
 ('us', 444183),
 ('DM', 326609),
 ('help', 253544),
 ('Please', 232561),
 ('We', 212200),
 ('Hi', 210674),
 ('get', 193941),
 ('please', 167897),
 ('Thanks', 165511)]

- 제일 많이 등장한 단어 10개를 most_common(10)을 이용해서 출력을 했다.
- 뒤에 나오는 수치는 등장 횟수이다.

In [28]:
common = set([w for (w,wc) in cnt.most_common(10)])
def remove_common(text):
  return ' '.join([word for word in text.split() if word not in common])
df['remove_common'] = df['remove_stopword'].apply([lambda text: remove_common(text)])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [29]:
df.head()

,text,lower_text,text_to_punct,remove_stopword,remove_common
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...,115712 send Private Message assist Just click ...
4,@sprintcare I did.,@sprintcare i did.,sprintcare I did,sprintcare I,sprintcare


#### 6. Removal of Rare Words
- 이는 앞서 가장 흔한 단어를 제거한 것과 방법이 비슷하다.

In [34]:
df.drop(["text_to_punct","lower_text", "remove_stopword"], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [36]:
from collections import Counter
cnt = Counter()
#Sol 1.
for text in df['remove_common'].values:
  for word in text.split():
    cnt[word] -= 1
rare = set([w for (w,wc) in cnt.most_common(10)])

#Sol 2.
#.most_common()을 하면 많이 사용된 순서로 나열하니까 slicing이 가능하다.
rare = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])

In [37]:
rare

{'115715',
 '115717',
 'Takeover',
 'httpstco0G98RtNxPK',
 'httpstcoU6ptkQa5Ik',
 'httpstcoV4yfrHR8VI',
 'nice—',
 'overmaybe',
 'unawa',
 'youSince'}

#### 7. Stemming
- stemming은 한글로 따지자면 어간, 품사등을 결정짓는 요소들을 제거하게 한다.
- 예를 들면 'walk'와 'walking'이라는 두 단어가 있다고 할 때에 stemming을 적용하면 'walk'로 바꿔주게 된다.
  - 그러나 간혹 제대로 구성되지 않은 단어로 바뀌는 경우가 있으니 이는 주의해 주어야 한다.
- 우리는 이를 ```porter stemmer```을 이용해서, 이도 마찬가지로 nltk모듈에서 import 해서 사용하면 된다.

In [43]:
df.head()

,text,remove_common
0,@115712 I understand. I would like to assist y...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,115712 send Private Message assist Just click ...
4,@sprintcare I did.,sprintcare


In [45]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def stem_word(text):
  return ' '.join([stemmer.stem(word) for word in text.split()])
df['stemmed'] = df['text'].apply(lambda text: stem_word(text))
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,text,remove_common,stemmed
0,@115712 I understand. I would like to assist y...,115712 understand would like assist would need...,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that,sprintcare propose,@sprintcar and how do you propos we do that
2,@sprintcare I have sent several private messag...,sprintcare sent several private messages one r...,@sprintcar I have sent sever privat messag and...
3,@115712 Please send us a Private Message so th...,115712 send Private Message assist Just click ...,@115712 pleas send us a privat messag so that ...
4,@sprintcare I did.,sprintcare,@sprintcar I did.


- 하지만 이렇게 하면 문제가 발생한다. 
- 위에서 주의해야 했던 부분인 실제 단어로서 존재하지 않는 형태로 바뀐 것이다.
   - 위에서 알아볼 수 있는 예로 propose를 존재하지 않는 단어인 propos로 바꾼 것을 볼 수 있다.
- 이를 해결하기 위해서 Lemmatation을 이용할 수 있다.
   - 이는 저장된 어원으로부터 존재하면 단어를 변형해주는 방법을 사용하기 때문에 시간은 오래 걸리더라도 훨씬 정확도 면에서 좋다고 할 수 있다.

In [ ]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
def lematize_words(text):
  return ' '.join([lemma.lemmatize(word) for word in text.split()])
df['lemma'] = df['text'].apply(lambda text: lematize_words(text))

In [ ]:
df.head()

- 그러나 문제는 ```WordNetLemmatizer```특성상 명사인지 동사인지 등의 품사를 지정해 주지 않으면 아래 코드의 결과처럼 변경이 제대로 안되는 경우가 발생한다는 것이다.

In [47]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
lemma.lemmatize('running')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


'running'

In [49]:
lemma.lemmatize('running', 'v')

'run'

In [56]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.pos_tag('running to forest'.split())

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('running', 'VBG'), ('to', 'TO'), ('forest', 'VB')]

In [58]:
from nltk.corpus import wordnet
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_word(text):
  pos_tagged_text = nltk.pos_tag(text.split())
  return ' '.join([lemma.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN))for (word, pos) in pos_tagged_text])
df['lemmatized'] = df['text'].apply(lambda text: lemmatize_word(text))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [59]:
df.head()

,text,remove_common,stemmed,lemmatized
0,@115712 I understand. I would like to assist y...,115712 understand would like assist would need...,@115712 I understand. I would like to assist y...,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that,sprintcare propose,@sprintcar and how do you propos we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,sprintcare sent several private messages one r...,@sprintcar I have sent sever privat messag and...,@sprintcare I have send several private messag...
3,@115712 Please send us a Private Message so th...,115712 send Private Message assist Just click ...,@115712 pleas send us a privat messag so that ...,@115712 Please send u a Private Message so tha...
4,@sprintcare I did.,sprintcare,@sprintcar I did.,@sprintcare I did.


### 이 이후부터는 파이썬의 정규 표현식 기능인 ```import re```의 기능이 매우 중요하다.


#### 8. Removal of URLS
- 문자열 중간 중간에 URL이 존재할 수 있는데, 이를 나중에 분석을 위해서 제거하는 것이 필수적이다.

In [66]:
def remove_url(text):
  url_pattern = re.compile(r'https?://\S+|www\.\S+')
  return url_pattern.sub(r'',text)
text = "Please refer to link http://lnkd.in/ecnt5yC for the paper"
remove_url(text)

'Please refer to link  for the paper'

In [68]:
text = "Want to know more. Checkout www.h2o.ai for additional information"
remove_url(text)

'Want to know more. Checkout  for additional information'

#### 9. Removal of HTML tags
- 우리는 다양한 사이트에서 웹데이터를 스콜링해서 이용하게 될 것이다.
- 이때 많이 사용하는 것이 BeautifulSoup 모듈이다.
- 한번 HTML 형태로 크롤링을 해서 이 데이터에 있는 태그를 없애보는 것 까지 진행해 보자.


사실 크롤링 과정은 간단하다. 그렇기에 이후의 처리가 상당히 중요하다.

In [69]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://entertain.naver.com/home')

file = response.text
print(file)

<!DOCTYPE html>
<html lang="ko">
<head>
	<title id="browse_title">네이버 TV연예</title>
	
	<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<script type="text/javascript" charset="utf-8">
var doc = document.documentElement;
doc.setAttribute('data-useragent', navigator.userAgent);
</script>
<link rel="shortcut icon" type="image/x-icon" href="https://ssl.pstatic.net/static.news/image/news/2014/favicon/favicon.ico" />
<script type="text/javascript" src="https://static-entertain.pstatic.net/pc/resources/20200813_054005/js/infra/jindo/jindo.desktop.ns.min.js"></script>
<script type="text/javascript" src="https://static-entertain.pstatic.net/pc/resources/20200813_054005/js/infra/common/enter.define.js"></script>
<script type="text/javascript" src="https://static-entertain.pstatic.net/pc/resources/20200813_054005/js/infra/image/imageError.js"></script>
<script type="text/javascript">
document.domain = 'naver.com';
</script>
<script type="text/javascript">
(function () {

In [77]:
#네이버 연예면을 크롤링했더니 나온 HTML파일 형식이 현재 텍스트로 바뀌어서 출력된 모습을 확인할 수 있다.
#이 속에 담긴 정보를 더 잘 활용하기 위해서 후처리를 진행해 보자
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="ko">
<head>
<title id="browse_title">네이버 TV연예</title>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script charset="utf-8" type="text/javascript">
var doc = document.documentElement;
doc.setAttribute('data-useragent', navigator.userAgent);
</script>
<link href="https://ssl.pstatic.net/static.news/image/news/2014/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<script src="https://static-entertain.pstatic.net/pc/resources/20200813_054005/js/infra/jindo/jindo.desktop.ns.min.js" type="text/javascript"></script>
<script src="https://static-entertain.pstatic.net/pc/resources/20200813_054005/js/infra/common/enter.define.js" type="text/javascript"></script>
<script src="https://static-entertain.pstatic.net/pc/resources/20200813_054005/js/infra/image/imageError.js" type="text/javascript"></script>
<script type="text/javascript">
document.domain = 'naver.com';
</script>
<script type="text/javascript">
(function () {
	

- requests.get()를 이용해서 불러온 데이터를 .text로 변형하나 위와 같이 하나 결과는 같다.

In [92]:
for tag in soup.select('a[class=title]'):
    print(tag.text)

'VMA2020' 방탄소년단, 베스트 그룹·베스트 K팝 수상.."아미 고마워"
'1호' '김학래♥︎' 임미숙, 공황장애 눈물→ 김지혜 "내가 입 열면 이혼1호 될수도"
'청춘기록' 측 "박보검X매니저 신동미, 완벽 시너지로 재미…
[단독]'뭉찬' CP "추성훈 曰 '김동현보다 잘해..타고난…
양치승 "황석정, 50대에 피트니스 대회…서울대 가는것보다 …
[속보] 
의정부지검 서동재 검사 실종
[차은우]
완벽한 화보의 완성은
★차은우…
감빵생활 깜짝 공개된 배우
트롯유망주'트롯전국체전'에…
세자로 데뷔해
세자 15년차인, 주지훈
[이루비, 다영] 신개념 만찢 비주얼 배…
[MCND]
‘nanana&r…
[안소희] 너무 종아!
'한승우 누나' 한선화, 고추장 수영복도 소화하는 '여신 미모'
레드벨벳 슬기, 장꾸 매력..우산 든 귀요미
'미코 眞' 김세연, 9컷으로 보니 더 예쁘네..내추럴 미모
'비밀의 숲2' 윤세아 '메이크업 삭제신' 레전드 명장면 탄생
'슈돌' 하영, 승재·하오 계보 잇는 언어 천재 '심장 폭격'
'임신' 최희 "아빠 생신상 차려드렸다..요알못이라 중간에 위기"[★SHOT!]
'허준'·'대장금' 신국, 지병으로 별세…향년 74세
'복면가왕' 유성은, 압도적 가창력+깊은 감성.."널리 노래 부를 수 있는 가수 되고파"
"사람들이 저를 바보로 알아요"..'물어보살' 40만 유튜버 덕자의 속사정
블랙핑크 제니, 힙한 언더웨어 스타일링…눈빛에 숨멎
윌벤져스 울린 최진혁, 이토록 친근한 좀비라니
경리, 남친 정진운 보라고 올렸나..이 몸매 美쳤다
'1호가' 임미숙 "10년간 공황장애 투병"…스튜디오 눈물바다 [종합]
'뉴 뮤턴트', 아티스트 캐릭터 포스터…마블 新 돌연변이들 [N컷]
김소영 아나, 동글동글 귀요미상 미녀..오상진은 좋겠네
미씽: 그들이 있었다
선을 넘는 녀석들 리턴즈
집사부일체
트레인
서울촌놈
복면가왕
코미디빅리그
길정우, 평소와 다른 현쥬니 행동에 ‘의심’
이재황, 김형범×길정우 관계에 大 충격!!
“말도 안 돼” 문보령, 김형범 친아

- 이제 태그를 제거해 보면된다.

In [93]:
def remove_html(text):
  html_pattern = re.compile('<.*?>')
  return html_pattern.sub(r'',text)

In [97]:
remove_html(soup.text)

'\n\n\n네이버 TV연예\n\n\n\nvar doc = document.documentElement;\ndoc.setAttribute(\'data-useragent\', navigator.userAgent);\n\n\n\n\n\n\ndocument.domain = \'naver.com\';\n\n\n(function () {\n\tvar enterPcCookie = jindo.$Cookie().get("enter_pc");\n\tif (window.location.href.indexOf("viewType=pc") > 0 && (enterPcCookie == "false" || !enterPcCookie)) {\n\t\tjindo.$Cookie().set("enter_pc", "true", 0, "entertain.naver.com");\n\t} else if (jindo.$Agent().navigator().mobile && enterPcCookie != "true") {\n\t\tif ((window.location.href.indexOf("/photo/issue/") > 0) && (window.location.hash.indexOf("cid") > 0)) {\n\t\t\tvar hashValue = window.location.hash;\n\t\t\tvar obj = jindo.$S(hashValue).parseString()\n\t\t\tif (obj[\'#cid\'] != \'\' && obj[\'iid\']) {\n\t\t\t\twindow.location.href = "https://m.entertain.naver.com/entertain?id=" + obj[\'#cid\'] + "&imgId=" + obj[\'iid\']\n\t\t\t\t\t+ "&host=https://m.entertain.naver.com" + "&listUrl=https://m.entertain.naver.com";\n\t\t\t\treturn;\n\t\t\t}\n\t\

In [141]:
#또는 BeautifulSoup를 이용하면
soup = BeautifulSoup(response.content, 'html.parser')
def remove_html(text):
    return BeautifulSoup(text, "lxml").text
data = remove_html(soup.text)
data = re.findall('[\S]', str(data))

- 정규 표현식의 match만 이용하게 되면 그냥 처음부터 끝까지 탐색하고 없으면 멈추기 때문에 해당 정규 표현식의 패턴과 일치하는 것을 찾기 위해서는 findall을 사용해야 한다.

In [142]:
data[3:10]

['T', 'V', '연', '예', 'v', 'a', 'r']

In [123]:
text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>
"""
text = remove_html(text)
for i in text.split():
  print(i)

H2O
AutoML
Driverless
AI


#### 10. Chat Words Conversion
- 채팅하면서 사람들이 사용하는 축약어를 찾아서 실제로 의미를 파악할 수 있는 단어/문장으로 바꾸어 보자
- 이는 그냥 미리 축약어들을 저장해 놓고 비교해 가면서 찾으면 되는 것이다.
- 단, 모든 단어의 첫글자를 대문자로 바꾸어 준 뒤어 탐색하는 것이 축약어를 찾는데에는 중요하다.

#### 11. Spelling Correction
- 잘못된 맞춤법을 수정해 보자.
- 맞춤법을 수정하기 위해서 이용하려는 SpellChecker 함수는 Peter Norvig 이라는 사람이 만든 알고리즘을 토대로 설계된 모듈로, 해당 단어로부터 2개만큼 좌우로 떨어진 단어들을 살펴서 모든 조합을 word frequency list와 비교를 해서 더 잦은 빈도로 나타나는 단어의 조합을 정답으로 판단하여 맞춤법 오류를 고쳐주는 것이다.
- 아쉽지만 사용 가능한 언어는 영어밖에 없기는 하다.

In [ ]:
from pyspellchecker import SpellChecker

spell = SpellChecker()

def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
